REF: https://goldcopd.org/wp-content/uploads/2020/03/GOLD-2020-POCKET-GUIDE-ver1.0_FINAL-WMV.pdf
Need to get PFT date difference

exposure and lung function decline

PCA and cluster
- only exposure
    - SO2 TO PM25
    - PCA and K-means -> weights
- All

Correlation:
- PM PCs and exposure vs all PFT changes
- Highlight significant correlated PFT values and T0 and T1

Scatterplot for FEV1, FEV1/FVC, and highly correlated with PM exposure

FEV1/FVC post < 0.7
FEV1/FVC post >=0.7 Gold0
Grade classification 

Find out decreased PFT variables and highlight
compute 1 year change of all PFT variables

Random Forest
- all %pred post 




Correlation between 
- all averaged PM variables and changes in PFT
- changes in PFT and baseline PFT

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from sklearn.cluster import KMeans
from utility import *

sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)

In [ ]:
SUBJID = 'Subj'
ROOT_PATH = '/home/twkim/datascience/'
DATA_PATH = '/home/twkim/datascience/QCT_Pro_20200304.csv'

df_raw = pd.read_csv(DATA_PATH)
df_VZERO = df_raw.filter(regex='0$', axis=1)
df_subj_id = df_raw[SUBJID]

PM_col = [col for col in df_raw.columns if 'PM' in col]
PFT_col = df_raw.columns[12:59].values
DESC_col = ['Age_yr', 'Height_m', 'Weight_kg']

print('# of subjects: ' + str(len(subj_id)))

In [ ]:
# Preprocess
df_target = df_VZERO
df_target = remove_inf_and_nan(df_target)
df_target = remove_outlier(df_target,threshold=3.5)


In [ ]:
PCs, weights = PCA_2(df_target.values)

PC1_df_target = PCs.iloc[:,0].values
PC2_df_target = PCs.iloc[:,1].values
PC_weights_df_target = pd.DataFrame(weights, columns=df_target.columns, index=['PC1', 'PC2'])

In [ ]:
labels2_PC_df_target, centroids2_PC_df_target = K_means(PC1_df_target, PC2_df_target, n=2,plot=True,xlabel='PC1',ylabel='PC2')
labels3_PC_df_target, centroids3_PC_df_target = K_means(PC1_df_target, PC2_df_target, n=3,plot=True,xlabel='PC1',ylabel='PC2')
labels4_PC_df_target, centroids4_PC_df_target = K_means(PC1_df_target, PC2_df_target, n=4,plot=True,xlabel='PC1',ylabel='PC2')

# Correlation: PCs

In [ ]:
df_target['PC1'] = PC1_df_target
df_target['PC2'] = PC2_df_target
df_target[SUBJID] = subj_id
df_target = swap_col(df_target)

#fu01['Cluster_PCA, k=2'] = labels2_PC_fu01
#fu01['Cluster_PCA, k=3'] = labels3_PC_fu01
#fu01['Cluster_PCA, k=4'] = labels4_PC_fu01

df_target.head()

In [ ]:
# Pearson
p_corr_target = df_target.corr(method='pearson')
PCA_columns = p_corr_target.columns[:-2]
target_columns = p_corr_target.columns[-2:]
p_corr, p_values = show_cor_heatmap(p_corr_target,43,target_columns,target_columns,symmetric=True,return_pvalue=True)

In [ ]:
p_corr.to_excel(ROOT_PATH + 'output_Pro_p_corr.xlsx')
p_values.to_excel(ROOT_PATH + 'output_Pro_p_values.xlsx')